In [6]:
import numpy as np
import pandas as pd
import os
import json
from keras.models import Model
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Activation, Add, Dropout, Multiply
from keras.optimizers import Adam

In [7]:
path = os.getcwd() + '/ml1m/'

In [9]:
users = pd.read_csv(f'{path}preprocessed/users.txt', sep='\t', index_col="new_id")
users

,raw_dataset_id
new_id,
0,1
1,2
2,3
3,4
4,5
...,...
6035,6036
6036,6037
6037,6038


In [10]:
movies = pd.read_csv(f'{path}preprocessed/products.txt', sep='\t', index_col='new_id')
movies

,raw_dataset_id
new_id,
0,2
1,3
2,4
3,5
4,6
...,...
3025,3947
3026,3948
3027,3949


In [27]:
ratings = pd.read_csv(f'{path}preprocessed/ratings.txt', sep='\t')
ratings


,uid,pid,rating,timestamp
0,0,872,5,978300760
1,0,537,3,978302109
2,0,679,3,978301968
3,0,2606,4,978300275
4,0,1790,5,978824291
...,...,...,...,...
940958,6039,810,1,956716541
940959,6039,813,5,956704887
940960,6039,477,5,956704746
940961,6039,815,4,956715648


In [5]:
old_ratings = pd.read_csv(f'{path}/ratings.dat', sep="::", names=["user_id", "movie_id", "rating", "timestamp"], header=None, index_col = "user_id")
old_ratings

/var/folders/8j/vb8qq8v168531_7qk_cwrfyc0000gn/T/ipykernel_68281/3517905203.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  old_ratings = pd.read_csv(f'{path}/ratings.dat', sep="::", names=["user_id", "movie_id", "rating", "timestamp"], header=None, index_col = "user_id")


,movie_id,rating,timestamp
user_id,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [35]:
old_movies = pd.read_csv(f'{path}/movies.dat',  sep="::", names=["movie_id", "movie_name", "genre"], header=None, encoding='latin-1')
old_movies

/var/folders/8j/vb8qq8v168531_7qk_cwrfyc0000gn/T/ipykernel_21857/2639277948.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  old_movies = pd.read_csv(f'{path}/movies.dat',  sep="::", names=["movie_id", "movie_name", "genre"], header=None, encoding='latin-1')


,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [36]:
old_users = pd.read_csv(f'{path}/users.dat',  sep="::", names=["UserID","Gender","Age","Occupation","Zip-code"], header=None)
old_users

/var/folders/8j/vb8qq8v168531_7qk_cwrfyc0000gn/T/ipykernel_21857/3506029074.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  old_users = pd.read_csv(f'{path}/users.dat',  sep="::", names=["UserID","Gender","Age","Occupation","Zip-code"], header=None)


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [ ]:


from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

user_ids = user_encoder.fit_transform(ratings.userId)
movie_ids = movie_encoder.fit_transform(ratings.movieId)



In [ ]:
# train / val split
num_train = int(len(user_ids) * 0.8)
train_user_ids = user_ids[:num_train]
train_movie_ids = movie_ids[:num_train]
train_ratings = rating_df.rating.values[:num_train]
val_user_ids = user_ids[num_train:]
val_movie_ids = movie_ids[num_train:]
val_ratings = rating_df.rating.values[num_train:]

In [ ]:


num_users= user_ids.max()+1
num_movies = movie_ids.max() + 1



In [ ]:
train_ratings /= 5
val_ratings /= 5

In [ ]:


def get_ncf_model():
    user_inp = Input((1,))
    user_hidden = Embedding(input_dim=num_users, output_dim=64)(user_inp)
    user_hidden = Flatten()(user_hidden)
    
    item_inp = Input((1,))
    item_hidden = Embedding(input_dim=num_movies, output_dim=64)(item_inp)
    item_hidden = Flatten()(item_hidden)
    
    # element-wise multiplication
    mf_output = Multiply()([user_hidden, item_hidden])
    
    hidden = concatenate([user_hidden, item_hidden])
    hidden = Dense(128, activation='relu')(hidden)
    hidden = Dropout(0.2)(hidden)
    mlp_output = Dense(64, activation='relu')(hidden)    

    
    output = concatenate([mf_output, mlp_output])
    output = Dense(1, activation='sigmoid')(output)
    
    model = Model(inputs=[user_inp, item_inp], outputs=output)
    model.compile(loss='mse', optimizer='adam')
    return model



In [ ]:
model = get_ncf_model()
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
# early stopping wait for 1 epoch
callbacks = [EarlyStopping(patience=1)]

# train for 50 epochs
model.fit([train_user_ids, train_movie_ids], train_ratings,\
          validation_data=([val_user_ids, val_movie_ids], val_ratings), epochs=50, batch_size=128, callbacks=callbacks)

In [ ]:
def dcg_at_k(r, k):
    '''
    Compute DCG
    args:
        r: np.array, to be evaluated
        k: int, number of entries to be considered
    
    returns:
        dcg: float, computed dcg
        
    '''
    r = r[:k]
    dcg = np.sum(r / np.log2(np.arange(2, len(r) + 2)))
    return dcg


In [ ]:
# compute average ndcg for all users
def evaluate_prediction(predictions):
    '''
    Return the average ndcg for each users
    args:
        predictions: np.array user-item predictions
    returns:
        ndcg: float, computed NDCG
    '''
    ndcgs = []
    for target_user in np.unique(val_user_ids):
        # get movie ids and ratings associated with the target user.
        target_val_movie_ids = val_movie_ids[val_user_ids == target_user] 
        target_val_ratings = val_ratings[val_user_ids == target_user] 
        
        # compute ndcg for this user
        ndcg = ndcg_at_k(target_val_ratings[np.argsort(-predictions[val_user_ids == target_user])], k=30)
        ndcgs.append(ndcg)
    ndcg = np.mean(ndcgs)
    return ndcg

In [ ]:
# prediction & evalutation
predictions = model.predict([val_user_ids, val_movie_ids])
evaluate_prediction(predictions[:,0])